In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pickle
import os

MODEL_PATH = "friend_face_model.h5"
LABEL_PATH = "friend_labels.pkl"  
IMG_SIZE = 160
THRESHOLD = 0.60
NUM_CLASSES = 0 


print("--- SYSTEM CHECK ---")
if not os.path.exists(MODEL_PATH) or not os.path.exists(LABEL_PATH):
    print("ERROR: Files missing.")
    exit()

try:
    with open(LABEL_PATH, "rb") as f:
        le = pickle.load(f)
    class_names = le.classes_
    NUM_CLASSES = len(class_names)
    print(f" Labels Loaded: {class_names} (Total: {NUM_CLASSES})")
except Exception as e:
    print(f" Error loading labels: {e}")
    print(" pip install scikit-learn")
    exit()


print("Reconstructing Model Architecture")

def build_model():
    
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights=None 
    )
    base.trainable = False

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs * 255.0)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)
    return model

try:

    model = build_model()
    
   
    model.load_weights(MODEL_PATH)
    print("Model Weights Loaded Successfully.")
    
except Exception as e:
    print(f" Failed to load weights: {e}")
    print("Make sure NUM_CLASSES matches exactly what was trained!")
    exit()


xml_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(xml_path)

if face_cascade.empty():
    print(" Face Detector XML not found.")
    exit()

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print(" Webcam not found.")
    exit()

print("\n SYSTEM READY. PRESS 'q' TO QUIT.")

while True:
    ret, frame = cap.read()
    if not ret: break
    
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        # Draw Box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        try:
            # Prepare Face
            face_roi = frame[y:y+h, x:x+w]
            rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(rgb, (IMG_SIZE, IMG_SIZE))
            
            # Normalize (0-1)
            normalized = resized.astype(np.float32) / 255.0
            input_data = np.expand_dims(normalized, axis=0)
            
            # Predict
            preds = model.predict(input_data, verbose=0)
            idx = np.argmax(preds)
            conf = np.max(preds)
            
            # Show Result
            if conf > THRESHOLD:
                name = class_names[idx]
                color = (0, 255, 0)
                text = f"{name} ({int(conf*100)}%)"
            else:
                color = (0, 0, 255)
                text = f"Unknown ({int(conf*100)}%)"
            
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            
        except Exception as e:
            pass # Skip bad frames

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


--- SYSTEM CHECK ---
✅ Labels Loaded: ['21-45902-3' '22-46138-1' '22-46139-1' '22-46141-1' '22-46156-1'
 '22-46258-1' '22-46275-1' '22-46293-1' '22-46342-1' '22-46473-1'
 '22-46536-1' '22-46666-1' '22-46677-1' '22-46679-1' '22-46840-1'
 '22-46857-1' '22-46877-1' '22-46880-1' '22-46887-1' '22-46931-1'
 '22-46945-1' '22-46983-1' '22-47027-1' '22-47180-1 ' '22-47294-1'
 '22-47384-2' '22-47402-2' '22-47802-2' '22-47813-2' '22-47884-2'
 '22-47888-2' '22-47892-2' '22-47894-2' '22-47898-2' '22-47925-2'
 '22-47934-2' '22-47966-2' '22-47968-2' '22-48005-2' '22-48021-2'
 '22-48023-2 ' '22-48039-2' '22-48055-2' '22-48064-2' '22-48091-2'
 '22-48133-2 ' '22-48205-2' '22-48434-3' '22-48541-3' '22-48569-3'
 '22-48582-3' '22-48833-3' '22-49037-3' '22-49167-3' '22-49196-3'
 '22-49338-3' '22-49355-3' '22-49370-3' '22-49421-3' '22-49450-3'
 '22-49451-3' '22-49453-3' '22-49507-3' '22-49575-3' '22-49609-3'
 '22-49619-3' '22-49621-3' '22-49643-3' '22-49644-3' '22-49745-3'
 '22-49783-3' '22-49791-3' '22-4980

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


✅ Model Weights Loaded Successfully.

🚀 SYSTEM READY. PRESS 'q' TO QUIT.
